Perovskite modelling program - returning PSC stack from an input row (from perovskite database)

#### Changing our dataframe to have columns for individual elements, with a normalised coefficient and site for each
1. Initital cleaning of ions and coefficients

In [2]:

##### Final cleaned and split dataset 

import pandas as pd
import re

# Load the CSV file
file_path = r"C:\Users\c\Documents\PEROVSKITE PROJECT\PerovskiteML_project\Data\Perovsite database query.csv"
data = pd.read_csv(file_path)

# Define the columns to keep
columns_to_keep = [
    'Cell_stack_sequence', 'Cell_architecture',
    'Substrate_stack_sequence', 'Substrate_thickness',
    'ETL_stack_sequence', 'ETL_thickness', 'ETL_additives_compounds', 'ETL_additives_concentrations',
    'Perovskite_composition_a_ions', 'Perovskite_composition_a_ions_coefficients', 
    'Perovskite_composition_b_ions', 'Perovskite_composition_b_ions_coefficients',
    'Perovskite_composition_c_ions', 'Perovskite_composition_c_ions_coefficients', 
    'Perovskite_additives_compounds', 'Perovskite_additives_concentrations', 'Perovskite_thickness',
    'HTL_stack_sequence', 'HTL_thickness_list', 'HTL_additives_compounds', 'HTL_additives_concentrations',
    'Backcontact_stack_sequence', 'Backcontact_thickness', 
    'Backcontact_additives_compounds', 'Backcontact_additives_concentrations',
    'Add_lay_front', 'Add_lay_front_function', 'Add_lay_front_stack_sequence', 'Add_lay_front_thickness_list', 
    'Add_lay_front_additives_compounds', 'Add_lay_front_additives_concentrations',
    'Add_lay_back', 'Add_lay_back_function', 'Add_lay_back_stack_sequence', 'Add_lay_back_thickness_list', 
    'Add_lay_back_additives_compounds', 'Add_lay_back_additives_concentrations',
    'Encapsulation', 'Encapsulation_stack_sequence'
]

# Filter columns to keep only those that exist in the dataset
existing_columns = [col for col in columns_to_keep if col in data.columns]
data = data[existing_columns]

# Add an index column
data.reset_index(inplace=True)
data.rename(columns={'index': 'Index'}, inplace=True)

# Save the filtered dataset to a new CSV file
output_path = 'filtered_DatabaseMaterials_with_index.csv'
data.to_csv(output_path, index=False)
print("Filtered dataset with index saved as", output_path)

# Create a separate dataframe for ions and their coefficients
ion_columns = [
    'Perovskite_composition_a_ions', 'Perovskite_composition_a_ions_coefficients', 
    'Perovskite_composition_b_ions', 'Perovskite_composition_b_ions_coefficients',
    'Perovskite_composition_c_ions', 'Perovskite_composition_c_ions_coefficients'
]

ion_data = data[ion_columns]

# Save the unchanged ion data
output_path = 'ion_data_unchanged.csv'
ion_data.to_csv(output_path, index=False)
print("Unchanged ion data saved as", output_path)

# Function to clean molecule names
def clean_molecule_name(name):
    name = re.sub(r'[^a-zA-Z0-9\s\-()]+', ' ', name.strip())
    name = re.sub(r'\s+', ' ', name).strip()
    elements = [element for element in name.split() if element and not element.replace('.', '', 1).isdigit()]
    return elements

# Function to clean and convert coefficients to floats
def clean_and_convert_coefficient(coefficient):
    try:
        cleaned_coefficient = re.sub(r'[^0-9.eE-]', '', coefficient.replace(',', '').strip())
        return float(cleaned_coefficient) if cleaned_coefficient else 0.0
    except ValueError:
        return 0.0

# Function to normalize coefficients
def normalize_coefficients(cell):
    if pd.notna(cell):
        try:
            coefficients = [float(x.strip()) for x in re.split(r'[;|]', cell) if x.strip()]
            total_sum = sum(coefficients)
            return ';'.join(f"{val / total_sum:.3f}" for val in coefficients) if total_sum > 0 else cell
        except ValueError:
            return cell
    return cell

# Normalize coefficients in each column
coefficient_columns = [
    'Perovskite_composition_a_ions_coefficients', 
    'Perovskite_composition_b_ions_coefficients', 
    'Perovskite_composition_c_ions_coefficients'
]

for col in coefficient_columns:
    ion_data[col] = ion_data[col].apply(normalize_coefficients)

# Create a set of unique molecules and add new columns
unique_molecules = set()
for column_group in ['a', 'b', 'c']:
    ions_column = f'perovskite_composition_{column_group}_ions'
    coefficients_column = f'perovskite_composition_{column_group}_ions_coefficients'
    for _, row in ion_data.iterrows():
        ions, _ = clean_molecule_name(str(row[ions_column])), [clean_and_convert_coefficient(c) for c in str(row[coefficients_column]).split(';')]
        unique_molecules.update(ions)

# Create columns for each unique molecule and calculate proportions
for molecule in unique_molecules:
    ion_data[molecule] = 0.0

for index, row in data[coefficient_columns].iterrows():
    for column_group in ['a', 'b', 'c']:
        ions_column = f'perovskite_composition_{column_group}_ions'
        coefficients_column = f'perovskite_composition_{column_group}_ions_coefficients'
        ions = clean_molecule_name(str(row[ions_column]))
        coefficients = [clean_and_convert_coefficient(c) for c in str(row[coefficients_column]).split(';')]
        total_coeff = sum(coefficients) if sum(coefficients) != 0 else 1
        
        for ion, coeff in zip(ions, coefficients):
            ion_data.at[index, ion] += coeff / total_coeff


# Create a new column 'Layer_Type' to indicate if the row is multilayered or single-layered
ion_data['Layer Type'] = ion_data.apply(
    lambda row: 'Multi-layered Perovskite' if any('|' in str(row[col]) for col in ion_columns) else 'Single-layered Perovskite',
    axis=1
)

# Drop the original ion columns as before
ion_data = ion_data.drop(columns=ion_columns, errors='ignore')

# Save the modified DataFrame with the 'Layer_Type' column
output_file_path = 'modified_data_with_layer_type.csv'
ion_data.to_csv(output_file_path, index=False)
print("CSV file with layer type information modified and saved as:", output_file_path)

C:\Users\c\AppData\Local\Temp\ipykernel_22972\215694299.py:8: DtypeWarning: Columns (10,22,29,31,32,35,36,40,44,45,46,48,51,54,65,84,89,90,93,98,99,100,105,108,115,118,122,123,125,130,134,138,142,143,144,146,149,152,163,166,167,171,172,173,175,178,181,192,194,225,271,272,273,277,304,315,321,325,330,331,335,336,342,348,369,371,373,374,376,380,384,387,403,405,407,409) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


Filtered dataset with index saved as filtered_DatabaseMaterials_with_index.csv
Unchanged ion data saved as ion_data_unchanged.csv


C:\Users\c\AppData\Local\Temp\ipykernel_22972\215694299.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ion_data[col] = ion_data[col].apply(normalize_coefficients)


KeyError: 'perovskite_composition_a_ions'

In [ ]:
# for a given row / perovskite, return all non-zero element and coefficient entries

# Function to get non-zero cells for a specific row
def get_non_zero_cells(row_number):
    # Check if the row_number is valid
    if row_number < 0 or row_number >= len(ion_data):
        return "Invalid row number"
    
    # Get the specified row
    row = ion_data.iloc[row_number]

    # Find non-zero (non-empty) entries and their column names
    non_zero_cells = {col: value for col, value in row.items() if value != 0 and value != ''}

    return non_zero_cells

# Example usage
row_number = 1234 # Replace with the row number you want to check
result = get_non_zero_cells(row_number)
print(f"Non-zero entries in row {row_number}: {result}")

Non-zero entries in row 1234: {'Index': np.int64(1234), 'Cell_stack_sequence': 'SLG | FTO | TiO2-c | TiO2-mp | Perovskite | Spiro-MeOTAD | Au', 'Cell_architecture': 'nip', 'Substrate_stack_sequence': 'SLG | FTO', 'Substrate_thickness': nan, 'ETL_stack_sequence': 'TiO2-c | TiO2-mp', 'ETL_thickness': '20.0 | 150.0', 'ETL_additives_compounds': 'Li-TFSI; Mg(TFSI)2', 'ETL_additives_concentrations': nan, 'Perovskite_composition_a_ions': 'Cs; FA; MA', 'Perovskite_composition_a_ions_coefficients': '0.050;0.750;0.200', 'Perovskite_composition_b_ions': 'Pb', 'Perovskite_composition_b_ions_coefficients': '1.000', 'Perovskite_composition_c_ions': 'I', 'Perovskite_composition_c_ions_coefficients': '1.000', 'Perovskite_additives_compounds': 'KI', 'Perovskite_additives_concentrations': nan, 'Perovskite_thickness': 500.0, 'HTL_stack_sequence': 'Spiro-MeOTAD', 'HTL_thickness_list': nan, 'HTL_additives_compounds': 'Co; Li-TFSI; TBP', 'HTL_additives_concentrations': nan, 'Backcontact_stack_sequence': 'Au

Hussain paper replication - formatting

In [7]:
# Filter the unique molecules set to keep only the specified molecules
molecules_to_keep = {'I', 'Br', 'Pb', 'Sn', 'Cl', 'FA', 'MA', 'Cs'}
filtered_unique_molecules = unique_molecules.intersection(molecules_to_keep)

# Update the output to show the filtered unique molecules
print("Filtered Unique molecules identified:", filtered_unique_molecules)

# Drop all columns in data_cleaned that are not in molecules_to_keep
columns_to_drop = [col for col in ion_data.columns if col not in molecules_to_keep and col not in columns_to_keep]
ion_data.drop(columns=columns_to_drop, inplace=True)

# Update the output to show the remaining columns
print("Remaining columns after dropping:", ion_data.columns)

# Save the modified dataframe to a new CSV
output_file_path = 'hussain_molecules_file.csv'
ion_data.to_csv(output_file_path, index=False)

print("CSV file modified and saved as:", output_file_path)

Filtered Unique molecules identified: {'Cl', 'Pb', 'MA', 'Sn', 'FA', 'I', 'Cs', 'Br'}
Remaining columns after dropping: Index(['Sn', 'Cs', 'MA', 'I', 'Cl', 'Br', 'Pb', 'FA'], dtype='object')
CSV file modified and saved as: hussain_molecules_file.csv


Re-format data into a dataframe we can easily vectorise to enable embedding:

In [4]:
## for each row - the code should be able to return values split into 3 different columns: a list of elements in the crystal, a list of their respective coefficients and sites
# eg. for row 123: [MA, I, Pb, Br], [1,1,0.5,0.5], [a,b,c,c]

# use original dataset

df = pd.read_csv(r"C:\Users\c\Documents\PEROVSKITE PROJECT\PerovskiteML_project\Data\Perovsite database query.csv")

# replace all ; and | in ion columns data with ,
# Replace all occurrences of ';' and '|' with ',' in the ion columns, keeping original if not found

df[ion_columns] = df[ion_columns].map(
    lambda x: str(x).replace(';', ',').replace('|', ',') if pd.notna(x) else x
)

# Continue with other processing or save the DataFrame as needed

# print(df[ion_columns].head())


### create sites column

# Function to extract the site from the column name
def get_site(column_name):
    # Extract the site part from the column name
    site = column_name.split('_')[3]  # 'a', 'b', or 'c' will be at index 3
    return site

# # Create a new 'site' column for ions and coefficients
# # Loop through ion columns and assign the site based on the column name
# ion_columns = [col for col in df.columns if 'Perovskite_composition' in col and 'ions' in col]
# coefficient_columns = [col for col in df.columns if 'Perovskite_composition' in col and 'coefficients' in col]


# # For each ion column, create a new site column
# site_mapping = {}

# for ion_col, coeff_col in zip(ion_columns, coefficient_columns):
#     site = get_site(ion_col)  # Extract site from ion column
#     site_mapping[ion_col] = site
#     site_mapping[coeff_col] = site

# # Now apply the new site to a new 'site' column
# df['site'] = [site_mapping[col] for col in df[ion_columns].columns if 'ions' in col]

# print(df['site'])

# combine a b and c ions and coefficients in 1 column 

df['combined_ions'] = df.apply(lambda row: f"{row['Perovskite_composition_a_ions']},{row['Perovskite_composition_b_ions']},{row['Perovskite_composition_c_ions']}", axis=1)
df['combined_coefficients'] = df.apply(lambda row: f"{row['Perovskite_composition_a_ions_coefficients']},{row['Perovskite_composition_b_ions_coefficients']},{row['Perovskite_composition_c_ions_coefficients']}", axis=1)

print(df[['combined_ions', 'combined_coefficients']].head())

# Save the updated DataFrame to a CSV file
output_file = "combined_ions_dataset.csv"
df.to_csv(output_file, index=False)

print(f"DataFrame with combined columns saved to {output_file}")


C:\Users\c\AppData\Local\Temp\ipykernel_22972\2706600968.py:6: DtypeWarning: Columns (10,22,29,31,32,35,36,40,44,45,46,48,51,54,65,84,89,90,93,98,99,100,105,108,115,118,122,123,125,130,134,138,142,143,144,146,149,152,163,166,167,171,172,173,175,178,181,192,194,225,271,272,273,277,304,315,321,325,330,331,335,336,342,348,369,371,373,374,376,380,384,387,403,405,407,409) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\c\Documents\PEROVSKITE PROJECT\PerovskiteML_project\Data\Perovsite database query.csv")


  combined_ions combined_coefficients
0       Cs,Sn,I                 1,1,3
1   Cs,Sn,Br, I          1,1,0.3, 2.7
2   Cs,Sn,Br, I          1,1,1.5, 1.5
3   Cs,Sn,Br, I          1,1,2.7, 0.3
4      Cs,Sn,Br                 1,1,3
DataFrame with combined columns saved to combined_ions_dataset.csv


In [160]:

# Define the site mapping based on original column names
site_mapping = {
    'Perovskite_composition_a_ions': 'a',
    'Perovskite_composition_b_ions': 'b',
    'Perovskite_composition_c_ions': 'c',
    'Perovskite_composition_a_ions_coefficients': 'a',
    'Perovskite_composition_b_ions_coefficients': 'b',
    'Perovskite_composition_c_ions_coefficients': 'c'
}

# Function to assign site based on the original columns
def assign_site(row):
    # Split the ions and coefficients into lists
    ions = row['combined_ions'].split(',')
    coefficients = row['combined_coefficients'].split(',')
    
    # Create a list for sites to store the corresponding site for each ion/coeff
    sites = []
    
    # Iterate through the ions and coefficients
    for ion, coeff in zip(ions, coefficients):
        # Determine which site this ion and coefficient belong to
        if ion in df['Perovskite_composition_a_ions'].values:
            sites.append('a')
        elif ion in df['Perovskite_composition_b_ions'].values:
            sites.append('b')
        elif ion in df['Perovskite_composition_c_ions'].values:
            sites.append('c')
        else:
            sites.append('Unknown')  # If not found, mark as unknown

    # Return the sites as a comma-separated string
    return ','.join(sites)

# Apply this function to the dataframe
df['site'] = df.apply(assign_site, axis=1)

# Print the updated dataframe
print(df[['combined_ions', 'combined_coefficients', 'site']].head())


  combined_ions combined_coefficients           site
0       Cs,Sn,I                 1,1,3          a,b,c
1   Cs,Sn,Br, I          1,1,0.3, 2.7  a,b,c,Unknown
2   Cs,Sn,Br, I          1,1,1.5, 1.5  a,b,c,Unknown
3   Cs,Sn,Br, I          1,1,2.7, 0.3  a,b,c,Unknown
4      Cs,Sn,Br                 1,1,3          a,b,c
